# XGBoost Regressor training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **AjeetSingh Charan's Cluster** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/1459130898943874/s?orderByKey=metrics.%60val_rmse%60&orderByAsc=true)
- Navigate to the parent notebook [here](#notebook/1459130898943864) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _10.4.x-cpu-ml-scala2.12_

In [0]:
import mlflow
import databricks.automl_runtime

target_col = "price"

## Load Data

In [0]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("64d00849c1dd4f699361989d8444f74c", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

,host_is_superhost,cancellation_policy,instant_bookable,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,...,bathrooms_na,beds_na,review_scores_rating_na,review_scores_accuracy_na,review_scores_cleanliness_na,review_scores_checkin_na,review_scores_communication_na,review_scores_location_na,review_scores_value_na,price
0,f,flexible,f,1.0,Bayview,37.72001,-122.39249,House,Entire home/apt,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0
1,f,flexible,f,1.0,Bayview,37.73250,-122.39221,House,Private room,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,45.0
2,f,flexible,f,1.0,Bernal Heights,37.73905,-122.41269,Apartment,Private room,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128.0
3,f,flexible,f,1.0,Bernal Heights,37.74220,-122.42091,Guest suite,Private room,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
4,f,flexible,f,1.0,Bernal Heights,37.74552,-122.41195,Apartment,Entire home/apt,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`[]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["review_scores_value", "review_scores_checkin", "bathrooms_na", "review_scores_location", "instant_bookable", "bathrooms", "review_scores_rating_na", "beds_na", "host_is_superhost", "neighbourhood_cleansed", "beds", "review_scores_cleanliness_na", "host_total_listings_count", "number_of_reviews", "review_scores_communication", "latitude", "cancellation_policy", "property_type", "review_scores_checkin_na", "review_scores_accuracy_na", "review_scores_value_na", "review_scores_cleanliness", "bedrooms_na", "review_scores_location_na", "review_scores_accuracy", "minimum_nights", "longitude", "accommodates", "room_type", "bedrooms", "review_scores_communication_na", "review_scores_rating", "bed_type"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

In [0]:
transformers = []

### Numerical columns

Missing values for numerical columns are imputed with mean by default.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

num_imputers = []
num_imputers.append(("impute_mean", SimpleImputer(), ["accommodates", "bathrooms", "beds", "host_total_listings_count", "latitude", "longitude", "minimum_nights", "number_of_reviews", "review_scores_rating"]))

numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors="coerce"))),
    ("imputers", ColumnTransformer(num_imputers, sparse_threshold=0))
])

transformers.append(("numerical", numerical_pipeline, ["minimum_nights", "longitude", "beds", "accommodates", "host_total_listings_count", "number_of_reviews", "review_scores_rating", "bathrooms", "latitude"]))

### Categorical columns

#### Low-cardinality categoricals
Convert each low-cardinality categorical column into multiple binary columns through one-hot encoding.
For each input categorical column (string or numeric), the number of output columns is equal to the number of unique values in the input column.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

one_hot_imputers = []

one_hot_pipeline = Pipeline(steps=[
    ("imputers", ColumnTransformer(one_hot_imputers, sparse_threshold=0, remainder="passthrough")),
    ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore")),
])

transformers.append(("onehot", one_hot_pipeline, ["bathrooms_na", "bed_type", "bedrooms", "bedrooms_na", "beds_na", "cancellation_policy", "host_is_superhost", "instant_bookable", "neighbourhood_cleansed", "property_type", "review_scores_accuracy", "review_scores_accuracy_na", "review_scores_checkin", "review_scores_checkin_na", "review_scores_cleanliness", "review_scores_cleanliness_na", "review_scores_communication", "review_scores_communication_na", "review_scores_location", "review_scores_location_na", "review_scores_rating_na", "review_scores_value", "review_scores_value_na", "room_type"]))

In [0]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

### Feature standardization
Scale all feature columns to be centered around zero with unit variance.

In [0]:
from sklearn.preprocessing import StandardScaler

standardizer = StandardScaler()

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [0]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=921183954)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=921183954)

## Train regression model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/1459130898943874/s?orderByKey=metrics.%60val_rmse%60&orderByAsc=true)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
from xgboost import XGBRegressor

help(XGBRegressor)

Help on class XGBRegressor in module xgboost.sklearn:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  XGBRegressor(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'reg:squarederror', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost regression.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of gradient boosted trees.  Equivalent to number of boosting
 |          rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      learning_rate : Optional[float]
 |          Boosting learning rate (xgb's "eta")
 |      verbosity : Optional[int]
 |          The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |      objective : typing.Union[str, typing.Callable[[numpy.ndarray, numpy.ndarray], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]
 |          Specify the learning ta

In [0]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display='diagram')

xgb_regressor = XGBRegressor(
  colsample_bytree=0.23642498999383274,
  learning_rate=0.15774862656750194,
  max_depth=9,
  min_child_weight=2,
  n_estimators=268,
  n_jobs=100,
  subsample=0.6214874428665853,
  verbosity=0,
  random_state=921183954,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
    ("regressor", xgb_regressor),
])

# Create a separate pipeline to transform the validation dataset. This is used for early stopping.
pipeline = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("standardizer", standardizer),
])

mlflow.sklearn.autolog(disable=True)
pipeline.fit(X_train, y_train)
X_val_processed = pipeline.transform(X_val)

model

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['review_scores_value',
 'review_scores_checkin', 'bathrooms_na',
 'review_scores_location',
 'instant_bookable', 'bathrooms',
 'review_scores_rating_na', 'beds_na',
 'host_is_superhost',
 'neighbourhood_cleansed', 'beds',
 'review_scores_cleanliness_na',
 'host_total_listings_count',
 'number_of_reviews',
 'review_sc...
 learning_rate=0.15774862656750194,
 max_delta_step=None, max_depth=9,
 min_child_weight=2, missing=nan,
 monotone_constraints=None, n_estimators=268,
 n_jobs=100, num_parallel_tree=None,
 predictor=None, random_state=921183954,
 reg_alpha=None, reg_lambda=None,
 scale_pos_weight=None,
 subsample=0.6214874428665853, tree_method=None,
 validate_parameters=None, verbosity=0))]) ColumnSelector ColumnSelector(cols=['review_scores_value', 'review_scores_checkin',
 'bathrooms_na', 'review_scores_location',
 'instant_bookable', 'bathrooms', 'review_scores_rating_na',
 'beds_na', 'host_is_superhost', 'neighbourhood_cleansed',
 'beds', 'review_scores_cleanliness_na',
 'host_total_listings_count', 'number_of_reviews',
 'review_scores_communication', 'latitude',
 'cancellation_policy', 'property_type',
 'review_scores_checkin_na', 'review_scores_accuracy_na',
 'review_scores_value_na', 'review_scores_cleanliness',
 'bedrooms_na', 'review_scores_location_na',
 'review_scores_accuracy', 'minimum_nights', 'longitude',
 'accommodates', 'room_type', 'bedrooms', ...]) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('numerical',
 Pipeline(steps=[('converter',
 FunctionTransformer(func= at 0x7ff8e5548d30>)),
 ('imputers',
 ColumnTransformer(sparse_threshold=0,
 transformers=[('impute_mean',
 SimpleImputer(),
 ['accommodates',
 'bathrooms',
 'beds',
 'host_total_listings_count',
 'latitude',
 'lo...
 'property_type', 'review_scores_accuracy',
 'review_scores_accuracy_na',
 'review_scores_checkin',
 'review_scores_checkin_na',
 'review_scores_cleanliness',
 'review_scores_cleanliness_na',
 'review_scores_communication',
 'review_scores_communication_na',
 'review_scores_location',
 'review_scores_location_na',
 'review_scores_rating_na',
 'review_scores_value',
 'review_scores_value_na', 'room_type'])]) numerical ['minimum_nights', 'longitude', 'beds', 'accommodates', 'host_total_listings_count', 'number_of_reviews', 'review_scores_rating', 'bathrooms', 'latitude'] FunctionTransformer FunctionTransformer(func= at 0x7ff8e5548d30>) imputers: ColumnTransformer ColumnTransformer(sparse_threshold=0,
 transformers=[('impute_mean', SimpleImputer(),
 ['accommodates', 'bathrooms', 'beds',
 'host_total_listings_count', 'latitude',
 'longitude', 'minimum_nights',
 'number_of_reviews',
 'review_scores_rating'])]) impute_mean ['accommodates', 'bathrooms', 'beds', 'host_total_listings_count', 'latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'review_scores_rating'] SimpleImputer SimpleImputer() onehot ['bathrooms_na', 'bed_type', 'bedrooms', 'bedrooms_na', 'beds_na', 'cancellation_policy', 'host_is_superhost', 'instant_bookable', 'neighbourhood_cleansed', 'property_type', 'review_scores_accuracy', 'review_scores_accuracy_na', 'review_scores_checkin', 'review_scores_checkin_na', 'review_scores_cleanliness', 'review_scores_cleanliness_na', 'review_scores_communication', 'review_scores_communication_na', 'review_scores_location', 'review_scores_location_na', 'review_scores_rating_na', 'review_scores_value', 'review_scores_value_na', 'room_type'] imputers: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0, transformers=[]) remainder passthrough passthrough OneHotEncoder OneHotEncoder(handle_unknown='ignore') remainder [['review_scores_value', 'review_scores_checkin', 'bathrooms_na', 'review_scores_location', 'instant_bookable', 'bathrooms', 'review_scores_rating_na', 'beds_na', 'host_is_superhost', 'neighbourhood_cleansed', 'beds', 'review_scores_cleanliness_na', 'host_total_lis

In [0]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(experiment_id="1459130898943874", run_name="xgboost") as mlflow_run:
    model.fit(X_train, y_train, regressor__early_stopping_rounds=5, regressor__eval_set=[(X_val_processed,y_val)], regressor__verbose=False)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    xgb_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    xgb_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    xgb_val_metrics = {k.replace("val_", ""): v for k, v in xgb_val_metrics.items()}
    xgb_test_metrics = {k.replace("test_", ""): v for k, v in xgb_test_metrics.items()}
    display(pd.DataFrame([xgb_val_metrics, xgb_test_metrics], index=["validation", "test"]))

,mse,mae,r2_score,rmse,score
validation,69244.61504,84.634251,0.270484,263.143716,0.270484
test,129709.12944,95.646394,0.114850,360.151537,0.114850


In [0]:
# Patch requisite packages to the model environment YAML for model serving
import os
import shutil
import uuid
import yaml

None

import xgboost
from mlflow.tracking import MlflowClient

xgb_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], str(uuid.uuid4())[:8])
os.makedirs(xgb_temp_dir)
xgb_client = MlflowClient()
xgb_model_env_path = xgb_client.download_artifacts(mlflow_run.info.run_id, "model/conda.yaml", xgb_temp_dir)
xgb_model_env_str = open(xgb_model_env_path)
xgb_parsed_model_env_str = yaml.load(xgb_model_env_str, Loader=yaml.FullLoader)

xgb_parsed_model_env_str["dependencies"][-1]["pip"].append(f"xgboost=={xgboost.__version__}")

with open(xgb_model_env_path, "w") as f:
  f.write(yaml.dump(xgb_parsed_model_env_str))
xgb_client.log_artifact(run_id=mlflow_run.info.run_id, local_path=xgb_model_env_path, artifact_path="model")
shutil.rmtree(xgb_temp_dir)

## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = False

In [0]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, len(X_train.index)))

    # Sample a single example from the validation set to explain. Increase the sample size and rerun for more thorough results.
    example = X_val.sample(n=1)

    # Use Kernel SHAP to explain feature importance on the example from the validation set.
    predict = lambda x: model.predict(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="identity")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example)

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/6de0c22cf90345f584005e2af2b59cf5/model
